In [ ]:
# Cell 1: Installation & Setup
# IMPORTANT: Restart your kernel before running this cell if you see pyarrow errors!
import os

# Install libraries
# We force upgrade pyarrow to ensure compatibility
!git clone https://github.com/QingruZhang/AdaLoRA.git
!pip install ./AdaLoRA/loralib
!pip install transformers datasets evaluate accelerate scikit-learn bitsandbytes
!pip install --upgrade pyarrow 

print("Installation complete. If you get a pyarrow error below, RESTART KERNEL and run again.")

Cloning into 'AdaLoRA'...
remote: Enumerating objects: 1603, done.
remote: Counting objects: 100% (1603/1603), done.
remote: Compressing objects: 100% (781/781), done.
remote: Total 1603 (delta 818), reused 1601 (delta 816), pack-reused 0 (from 0)
Receiving objects: 100% (1603/1603), 4.20 MiB | 21.41 MiB/s, done.
Resolving deltas: 100% (818/818), done.
Processing ./AdaLoRA/loralib
  Preparing metadata (setup.py) ... done
  Created wheel for loralib: filename=loralib-0.1.0-py3-none-any.whl size=13657 sha256=53a3851f17eb88ec70c228cf9573eb4de95d6750b91f173f9a0a4bc04e4f4019
  Stored in directory: /tmp/pip-ephem-wheel-cache-wo3tvhxm/wheels/a4/32/19/31e636f5205565008b63240ad971590477bdbce7da08342170
Successfully built loralib
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 19.5 MB/s eta 0:00:00:00:0100:01
  Attempting

In [2]:
!pip install datasets
!pip install transformers
!pip install -U peft
!pip install evaluate
!pip install -U datasets==2.20.0 pyarrow==15.0.2 transformers==4.44.2 evaluate==0.4.2 --no-cache-dir
!pip install -q datasets evaluate accelerate scikit-learn pandas matplotlib
!pip install -U "transformers>=4.41" accelerate safetensors
!pip install -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 246.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 204.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 208.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 240.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 456.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 176.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 209.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 135.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 113.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [1]:
import os
import time
import warnings
from typing import Optional
import torch
import numpy as np
import evaluate
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    logging as hf_logging,
)
from peft import AdaLoraConfig, get_peft_model, TaskType

In [2]:
os.environ.setdefault("CUDA_VISIBLE_DEVICES", "0")
os.environ["WANDB_DISABLED"] = "true"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore")
hf_logging.set_verbosity_error()

In [3]:
def load_dataset_with_retry(path, name=None, max_retries=None, wait=2):
    attempt = 0
    while True:
        try:
            ds = load_dataset(path, name) if name else load_dataset(path)
            print(f"✅ Loaded dataset '{path}'" + (f"/'{name}'" if name else ""))
            return ds
        except Exception as e:
            attempt += 1
            wait_time = wait * min(5, attempt)
            print(f"[Attempt {attempt}] Failed: {e}")
            time.sleep(wait_time)
            if max_retries and attempt >= max_retries:
                raise

In [4]:
def clean_text(s: Optional[str]) -> str:
    return "" if s is None else " ".join(str(s).strip().split())

def build_one_shot_demo(example: dict) -> str:
    return (
        f"Example:\n"
        f"Question: {clean_text(example['question'])}\n"
        f"Context: {clean_text(example['context'])}\n"
        f"Answer (Yes/No): {example.get('label_text', 'Yes')}\n\n"
    )

ONE_SHOT_EXAMPLE = {
    "question": "Who wrote Hamlet?",
    "context": "Hamlet was written by William Shakespeare.",
    "label_text": "Yes",
}

In [5]:
model_checkpoint = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
output_dir = "./tinyllama-qnli-adalora"

batch_size = 16
eval_batch_size = 32
num_train_epochs = 3
max_length = 128
learning_rate = 2e-5
weight_decay = 0.01
rank = 8
seed = 42

torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

Device: cuda


In [6]:
print("📚 Loading GLUE/QNLI...")
dataset = load_dataset_with_retry("glue", "qnli")

📚 Loading GLUE/QNLI...


Generating train split:   0%|          | 0/104743 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5463 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5463 [00:00<?, ? examples/s]

✅ Loaded dataset 'glue'/'qnli'


In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(examples, add_demo=False):
    demo = build_one_shot_demo(ONE_SHOT_EXAMPLE) if add_demo else ""

    prompts = []
    for q, c in zip(examples["question"], examples["sentence"]):
        prompt = (
            demo
            + f"Question: {clean_text(q)}\n"
              f"Context: {clean_text(c)}\n"
              f"Answer (Yes/No):"
        )
        prompts.append(prompt)

    return tokenizer(prompts, truncation=True, max_length=max_length)

tokenized = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=["question", "sentence", "idx"],
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Map:   0%|          | 0/104743 [00:00<?, ? examples/s]

Map:   0%|          | 0/5463 [00:00<?, ? examples/s]

Map:   0%|          | 0/5463 [00:00<?, ? examples/s]

In [8]:
tokenized = tokenized.rename_column("label", "labels")
tokenized.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

train_dataset = tokenized["train"].select(range(10_000))
eval_dataset = tokenized["validation"].select(range(1_500))

steps_per_epoch = (len(train_dataset) + batch_size - 1) // batch_size
total_steps = steps_per_epoch * num_train_epochs

In [9]:
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding="longest",
    pad_to_multiple_of=8,
)

In [10]:
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding="longest",
    pad_to_multiple_of=8,
)


In [11]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=2,
    id2label={0: "entailment", 1: "not_entailment"},
    label2id={"entailment": 0, "not_entailment": 1},
)

model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False
model.to(device)

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1

In [12]:
adalora_config = AdaLoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=rank,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    init_r=12,
    target_r=rank,
    tinit=int(0.1 * total_steps),
    tfinal=int(0.8 * total_steps),
    deltaT=max(1, int(0.04 * total_steps)),
    beta1=0.85,
    beta2=0.85,
    orth_reg_weight=0.5,
    total_step=total_steps,
)

model = get_peft_model(model, adalora_config)
model.print_trainable_parameters()

trainable params: 9,467,704 || all params: 1,043,984,338 || trainable%: 0.9069


In [13]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    return accuracy.compute(predictions=preds, references=labels)

In [16]:
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=eval_batch_size,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    num_train_epochs=num_train_epochs,
    eval_strategy="epoch",
    save_strategy="no",
    logging_steps=50,
    report_to="none",
    seed=seed,
    bf16=torch.cuda.is_available(),
    disable_tqdm=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [17]:
print("🚀 Training...")
trainer.train()

🚀 Training...


Epoch,Training Loss,Validation Loss,Accuracy
1,0.922500,0.876699,0.609333
2,0.826100,0.783927,0.672667
3,0.737600,0.730166,0.712000


TrainOutput(global_step=1875, training_loss=0.9186794392903646, metrics={'train_runtime': 7920.6254, 'train_samples_per_second': 3.788, 'train_steps_per_second': 0.237, 'total_flos': 1.9781125670025216e+16, 'train_loss': 0.9186794392903646, 'epoch': 3.0})

In [18]:
print("📊 Evaluation...")
print(trainer.evaluate())

📊 Evaluation...


{'eval_loss': 0.7301656603813171, 'eval_accuracy': 0.712, 'eval_runtime': 181.9638, 'eval_samples_per_second': 8.243, 'eval_steps_per_second': 0.258, 'epoch': 3.0}


In [19]:
print("💾 Saving model...")
trainer.save_model(output_dir)

💾 Saving model...
